# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import nltk
from nltk.corpus import stopwords
nltk.download(['punkt', 'wordnet','stopwords'])

In [ ]:
import os
import re
import time
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import warnings
warnings.filterwarnings('ignore')

from sklearn import preprocessing
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

from sqlalchemy import create_engine

In [ ]:
db_path = os.path.join(os.path.abspath('../data/processed/'), 'ResponseDatabase.db')
db_uri = 'sqlite:///{}'.format(db_path)

# load data from database
engine = create_engine(db_uri)
df = pd.read_sql_table('ResponseTable', con=engine)
df = df.set_index('id')
X = df['message']
y = df.iloc[:,3:]

In [ ]:
df.head()

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):
    #normalize - Remove punctuation characters
    regex = '[^a-zA-Z0-9]'
    detect_text = re.findall(regex, text)

    #tokenize - split text into words
    for list in text:
        list = text.replace(regex," ")
        tokens = word_tokenize(list)

    # Remove Stopwords
    words = [word for word in tokens if word not in stopwords.words('english')]

    #lemanize
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []

    for word in words:
        clean_tok = lemmatizer.lemmatize(word).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
# Random Forest Classifier - pipeline
pipeline_rf = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs=5)))
])

pipeline_lgb = Pipeline([
    ('tfidfvect', TfidfVectorizer(tokenizer=tokenize)),
    ('multiclassifier',MultiOutputClassifier(LGBMClassifier(n_jobs=5)))
])

In [ ]:
# Pipeline dictionary
pipeline_dict = {'pipeline_rf': pipeline_rf, \
                 'pipeline_lgb ': pipeline_lgb }

### 4. Train various pipelines
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def display_scores(model_dict, X_train, y_train):
    
    """
    Function for applying the cross validation score
    
    Parameters:
    
    model, X_train, y_train
    
    Returns:
    
    f1 - Scores and Average
    
    """
    
    # cross validation for F1 score
    results = {} 

    for pipename, pipevalue in pipeline_dict.items() :
        start_time = time.time()
        print ("Training pipeline : {} ...".format(pipename))
        scores = cross_val_score(pipevalue, X_train, y_train, scoring='f1_weighted', cv=5)
        results[pipename] = scores.mean()
        print ("Pipeline : {} F1 mean score {}".format(pipename, scores.mean()))
        time_taken = round(((time.time() - start_time) / 60),3)
        print("--- " + str(time_taken) + " minutes ---")
        
    return results

In [ ]:
pipeline_f1_score = display_scores(pipeline_dict, X_train, y_train)

In [ ]:
pipeline_f1_score

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
# Results and Best pipeline
best_pipline_name = max(f1_results, key=f1_results.get)

ax = pd.DataFrame.from_dict(f1_results, orient='index').plot.barh(figsize=(12,5))
ax.set_xlabel("CV F1 Mean Scores")
ax.set_ylabel("Pipelines")
ax.set_title('Pipeline Training Results - best results '+ best_pipline_name)
plt.show()

In [ ]:
# Train best pipline
best_pipline = pipeline_dict[best_pipline_name]
best_pipline
#best_pipeline.fit(X_train, y_train)

In [ ]:
# predict on test data
y_pred = best_pipeline.predict(X_test)

In [ ]:
# Print Classification Report
print(classification_report(y_test, y_pred, target_names= y_test.columns.to_list()))

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
myscoring = make_scorer(f1_score,average='weighted')

parameters = {
     'tfidfvect__ngram_range': ((1, 1), (1, 2)),
     'tfidfvect__max_df': (0.5, 0.75, 1.0),
     'tfidfvect__max_features': (None, 100, 500, 2000)
#     'classifierchain__order' : 'random',
#     'classifierchain__base__estimator__learning_rate': [0.01, 0.1 , 0.5, 0,8],
#     'classifierchain__base__estimator__max_depth': [5, 10, 20],
# #    'multiclassifier__num_leaves': 50,
# #      'clf__min_samples_leaf': [1, 2, 4],
# #      'clf__min_samples_split': [2, 5, 10],
#     'classifierchain__baser__estimator__n_estimators': [100, 200, 400, 800 , 1000] 
    }

# create grid search object
search = RandomizedSearchCV(best_pipline, parameters, scoring=myscoring, n_jobs=-1)

result = search.fit(X_train, y_train)

# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)



In [ ]:
#Print grid search CV results
means = search.cv_results_['mean_test_score']
stds = search.cv_results_['std_test_score']
params = search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    display("param search results -> %f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# Best Model with tuned parameters
set_config(display='text')

best_model_tuned = search.best_estimator_
best_model_tuned

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# Print Classification Report
predictions = best_model_tuned.predict(X_test)
print(classification_report(y_test, predictions, target_names= y_test.columns.to_list()))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
import pickle

model_dir = '../models/'

#Get the date to time stamp model
today = datetime.now()
timestamp = today.strftime("%b-%d-%y-%Hh%M")

model_file = best_pipline_name+'-'+timestamp+'.pkl'
print("Saving model {} to directory {}".format(model_file,model_dir))
pickle.dump(best_model_tuned, open(model_dir+model_file, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.